In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import pandas as pd
import glob
import data_utils_celeba


import numpy as np
import pandas as pd
import os

In [2]:
# Root directory for dataset
dataroot = '/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Deep Learning/Projects/Algorithmic fairness/Data/'

# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

In [ ]:
attributes = pd.read_csv(dataroot+'Anno/list_attr_celeba.txt', sep=" ")
attributes.drop('Unnamed: 41',axis=1, inplace=True)
partition = pd.read_csv(dataroot+'Eval/list_eval_partition.txt', sep=" ", header=None, names=['im_id','partition'])
matched = attributes.set_index('im_id').join(partition.set_index('im_id'))

In [ ]:
train = matched[matched['partition']==0]
valid = matched[matched['partition']==1]
test = matched[matched['partition']==2]
train.to_csv('train_full.csv')
valid.to_csv('valid_full.csv')
test.to_csv('test_full.csv')

In [ ]:
len(test)

In [ ]:
pwd

In [3]:
IMAGE_SHAPE = (178,218)
# Paths to data
TRAIN_PATH = "train_full.csv" 
VALID_PATH = "valid_full.csv" 
TEST_PATH = "test_full.csv" 
IMAGE_PATHS = glob.glob('/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Deep Learning/Projects/Algorithmic fairness/Data/celeba/*.jpg')
TARGET_COL = 'Male'

# train holds both X (input) and t (target/truth)
data = data_utils_celeba.load_data(train_path=TRAIN_PATH,
                            valid_path = VALID_PATH,
                            test_path=TEST_PATH,
                            image_paths=IMAGE_PATHS,
                            target_col = TARGET_COL,
                            image_shape=IMAGE_SHAPE[:2])

Loading training data
	0 of 162770
	10000 of 162770
	20000 of 162770
	30000 of 162770
	40000 of 162770
	50000 of 162770
	60000 of 162770
	70000 of 162770


KeyboardInterrupt: 

In [ ]:
data

In [ ]:
import pickle

## Dump the data into a pickle file
#
with open('celeba_dataset.pickle', 'wb') as f:
    pickle.dump(data, f)
    
## Load the data from a pickle file
#with open('Kaggle/data.pickle', 'rb') as f:
#    data = pickle.load(f)

In [ ]:
NUM_CLASSES = 2

dummy_batch_gen = data_utils_celeba.batch_generator(data, batch_size=batch_size, num_classes=NUM_CLASSES, num_iterations=5e3, num_features=40, seed=42)

In [ ]:
train_batch = next(dummy_batch_gen.gen_train())
valid_batch, i = next(dummy_batch_gen.gen_valid())
test_batch, i = next(dummy_batch_gen.gen_test())

In [ ]:
train

In [ ]:
plt.imshow(np.transpose(vutils.make_grid(torch.Tensor(valid_batch['images'][1]), padding=2, normalize=True).cpu(),(0,1,2)))